# DATATTACK - Proteção Civil

 - Catarina Mendes
 - Carlos Faria
 - Nicholas Hopf
 - Rodrigo Gomes

---

This project aims to provide insight about issues targeted by the portuguese governmental agency "Proteção Civil". Those issues will be called "occurrences" in this notebook.

The scope of our work not only provides insight about past occurrences and the correlation between variables collected in each situation, but also aims to predict the resources needed to effectively solve a future occurrence.

At the end, we deployed a web application that serves as an interface for a relevant subset of the models developed in this project, making them freely available for every citizen and government agent to consult. We believe that data transparency can empower the public, so we made an effort to display a simple but functional application.

---

# CRISP-DM
CRISP-DM was the methodology that guided our action plan and allowed us to maintain a consistent workflow throughout the project.

At first, we made sure that our understanding about the actions performed by "Proteção Civil" was solid, so our following analysis and intervention proposals could be better grounded. After some research, we could get a sense on the economical and social impacts that an effective action could have.

Then we looked at the dataset provided. After some discussion about its quantitative and qualitative aspects, we also gathered another dataset with georeferenced information about each Portuguese administrated subdivision, so conversion between geographical coordinates and regions could be efficiently performed and visualized.

After that, we proceeded to prepare the provided dataset by removing invalid data and dividing the `Natureza` field into more columns based on the hierarchy involved in its inherent structure.


In [63]:
# Import necessary libraries and methods
import pandas as pd
import numpy as np

In [64]:

def datetime_to_hours_int(x):
    return [i.total_seconds()/3600 for i in x]

def clean_data(df, remove_false_alarms=True) -> pd.DataFrame:
    print('df initial length: ', len(df))

    data_clean = df[df['Natureza'] != '1']
    data_clean = data_clean[data_clean['EstadoOcorrencia'] != '2']
    data_clean = data_clean[data_clean['Distrito'] != '0']
    data_clean = data_clean[data_clean['Concelho'] != '0']
    data_clean = data_clean[data_clean['Numero'] != """\t\t\t\t\t\t\t\t"""]
    data_clean = pd.concat([data_clean[data_clean['EstadoOcorrencia'] == 'Encerrada'], data_clean[data_clean['EstadoOcorrencia'] == 'Falso Alarme']])

    # We replace the "," with "." to facilitate processing
    data_clean['Latitude'] = pd.to_numeric(
        data_clean['Latitude'].str.replace(',', '.')
    )
    data_clean['Longitude'] = pd.to_numeric(
        data_clean['Longitude'].str.replace(',', '.')
    )    

    data_clean['DataOcorrencia'] = pd.to_datetime(data_clean['DataOcorrencia'], format='%d/%m/%Y %H:%M:%S')
    data_clean['DataFechoOperacional'] = pd.to_datetime(data_clean['DataFechoOperacional'], format='%d/%m/%Y %H:%M:%S')

    if remove_false_alarms:
        data_clean = data_clean[data_clean['EstadoOcorrencia'] != 'Falso Alarme']

    data_clean.dropna(inplace=True)
    print('Number of occurrences: ', len(data_clean))

    data_clean['hh'] = np.multiply(
        (data_clean['NumeroOperacionaisAereosEnvolvidos'] + data_clean['NumeroOperacionaisTerrestresEnvolvidos']),
        datetime_to_hours_int(data_clean['DataFechoOperacional'] - data_clean['DataOcorrencia'])
    )

    return data_clean

# Define a function to extract category from a string
def _extract_category(string, cat_index):
    if string == "nan" or any(str.isdigit(c) for c in string):
        return
    else:
        parts = string.split("/", 4)
        return parts[cat_index-1].strip()


# Reformulate categories based on functions defined
def parse_categories(data_clean:pd.DataFrame):
    data_clean['category1'] = data_clean['Natureza'].astype(str).apply(lambda x: _extract_category(x,cat_index=1))
    data_clean['category2'] = data_clean['Natureza'].astype(str).apply(lambda x: _extract_category(x,cat_index=2))
    data_clean['category3'] = data_clean['Natureza'].astype(str).apply(lambda x: _extract_category(x,cat_index=3))
    
    #Consolidate categories with little statistical relevance
    relevance_threshold = 100
    category_counts = data_clean['category3'].value_counts()
    filtered_categories = category_counts[category_counts <= relevance_threshold].index.tolist()
    # Replace the filtered categories with a new "Other" category
    data_clean['category3'] = data_clean['category3'].replace(
        to_replace=filtered_categories,
        value='Outras ocorrências'
    )
    
    return data_clean



We also created a new metric called "men-hour", to measure the effort of "Proteção Civil" personnel during each occurrence.

After the dataset preparation, we performed several techniques to visualize the correlation between each variable, if existent, and performed a simple linear regression to verify how high is the correlation between classes of occurrences and their district of origin.


In [ ]:
import pandas as pd
from sklearn.feature_extraction.text import HashingVectorizer
from sklearn.linear_model import LinearRegression
from sklearn.feature_extraction import FeatureHasher

# Read the database .CSV
occurrences = pd.read_csv(
    "Data/anpc-2016.csv", sep=",", on_bad_lines="skip", low_memory=False
)

# Get relevant and error-free data
occurrences = parse_categories(clean_data(occurrences))

# Treat the 
def hashed_reg(occurrences):
    # Encode the categorical variables using feature hashing with HashingVectorizer
    num_districts = occurrences.nunique()["Distrito"]
    hash_size_increase = 2.5  # Increase number of features to avoid hash collision
    district_vectorizer = HashingVectorizer(n_features=num_districts, norm=None, alternate_sign=False)
    X_cat = district_vectorizer.fit_transform(occurrences["Distrito"])

    num_categories = occurrences.nunique()["category2"]
    hash_size_increase = 2.5  # Increase number of features to avoid hash collision
    category_vectorizer = HashingVectorizer(n_features=num_categories, norm=None, alternate_sign=False)
    X_cat2 = category_vectorizer.fit_transform(occurrences["category2"])

    # Combine the encoded categorical variables and the numerical variable
    X = pd.concat(
        [
        pd.DataFrame(X_cat.toarray()), 
        pd.DataFrame(X_cat2.toarray()),
        pd.DataFrame(datetime_to_hours_int(occurrences['DataOcorrencia'] - min(occurrences['DataOcorrencia'])))
        ], axis=1
    )

    # Fit a linear regression model to the data
    model = LinearRegression()
    model.fit(X, occurrences['hh'])

    return model.predict(X)

def hashed_reg_2(occurrences):
    num_districts = occurrences.nunique()["Distrito"]
    hash_size_increase = 2.5  # Increase number of features to avoid hash collision
    h = FeatureHasher(n_features=int(hash_size_increase * num_districts), input_type="string")
    return h.transform(list(occurrences["Distrito"]))

hashed_reg(occurrences)


Alongside the analysis efforts dedicated to our primary data source, we also created a simple script that loads georreferenced data and associates input coordinates into portuguese political divisions. This script was used later be used in our web application.


In [ ]:
import geopandas as gpd
from shapely.geometry import Point
import matplotlib.pyplot as plt
%matplotlib inline

# shp -> shapefile (geometry)
# dbf -> database (extra info)
# prj - projection
# shx - index

# Get shape of municipalities and parishes
portugal_municipalities = gpd.read_file("./georeferencing/data_concelhos/concelhos.shp")

# Remove islands
portugal_municipalities = portugal_municipalities[portugal_municipalities.NAME_1 != "Azores"]
portugal_municipalities = portugal_municipalities[portugal_municipalities.NAME_1 != "Madeira"]

# Get point defined by user (this will be used later in the API)
lat, lon = 40.9800516, -8.202641
user_point = Point(lon, lat)


def get_municipality(point: Point):
    for index, row in portugal_municipalities.iterrows():
        polygon = row['geometry']
        # Check if the point is inside the polygon
        if polygon.contains(point):
            # Return the index or other properties of the polygon
            return {
                "district": row.NAME_1,
                "municipality": row.NAME_2,
                "municipality_gdf": gpd.GeoDataFrame([row], crs=portugal_municipalities.crs)
            }


def plot_point_in_map(point, municipality_gdf):
    point_df = gpd.GeoDataFrame(geometry=[point])
    pt_map = portugal_municipalities.plot()
    municipality_gdf.plot(ax=pt_map, color='yellow')
    point_df.plot(ax=pt_map, color='red', markersize=10)
    plt.axis('off')
    plt.show()


municipality_data = get_municipality(point=user_point)
plot_point_in_map(user_point, municipality_data["municipality_gdf"])

As mentioned before, we visualized data correlation in several ways. The script below saved several figures that would later be used in our report to illustrate the prevalence of each class of occurrence in each district.

In [ ]:
# Imports
import pandas as pd
import os

#Parses the data and makes list of lists based on unique values for the Districts columns
def occurrence_frequency(df: pd.DataFrame, categorylevel: str, geoscope: str, datasetisclean = False):
    
    if not datasetisclean:
        df = clean_data(df)
    
    geoscopef = geoscope
    geo_list = []
    abreviation_dict = {'Assistência e Prevenção a actividades humanas' : 1, 'Comprometimento total ou parcial de segurança, serviços ou estruturas': 2,
                        'Incêndios Urbanos ou em Área Urbanizável':3, 'Incêndios em Equipamento e Produtos':4}

    for row1 in df[geoscopef].unique():
        cat_list = []
        i = 0
        #Parses the Districts columns to count occurrences
        for row2 in df[geoscopef]:
            if row1 == row2:
                category = df[categorylevel].iloc[i]
                #Abbreviates some categories for better visualization graphs
                if category in abreviation_dict:
                    value = abreviation_dict[category]
                    if value == 1:
                        category = "Assist e Prev a actv humanas"
                    elif value == 2:
                        category = "Comp total ou parcial de seg"
                    elif value == 3:
                        category = "Incêndios urbanos"
                    elif value == 4:
                        category = "Incêndio em equipm"
                #Verifies if category is in list and adds to occurrence count if it does, adds category to list if it doesn't
                if category in cat_list:
                    cat_list.index(category)
                    cat_list[cat_list.index(category)+1] = cat_list[cat_list.index(category)+1] + 1
                else:
                    cat_list.append(category)
                    cat_list.append(1)
            i = i + 1
        geo_list.append(cat_list)
        

    # Transform the elements in each sublist into tuples of two
    list_of_tuples = [[(lst[2*i], lst[2*i+1]) for i in range(round(len(lst)/2-1))] for lst in geo_list]

    #Define a counter and makes relation between occurrences and Districts for visualization of data
    j = 0
    for row1 in df[geoscopef].unique():
        list_of_tuples[j] = sorted(list_of_tuples[j], key = lambda x: (x[0]))
        # Separate the x and y values into two lists
        x_values = [x[0] for x in list_of_tuples[j]]
        y_values = [x[1] for x in list_of_tuples[j]]
        # Plot the data
        plt.bar(x_values, y_values)
        plt.title("Breakdown of occurrences for" + " " + str(row1))
        plt.xlabel("occurrence type")
        plt.xticks(fontsize=8,rotation=45,ha='right')
        plt.ylabel("Number of occurrences")
        plt.tight_layout()
        plt.savefig(os.path.join('.','images',f'{row1}_occurrences_{geoscope}_{categorylevel}.png'))
        plt.clf()
        plt.cla()
        j = j+1
        
    return list_of_tuples

As well as visualizing the most prevalent issue in each administrative subdivision on a map.

In [ ]:
import pandas as pd
import geopandas as gpd
import matplotlib.pyplot as plt
%matplotlib inline
from matplotlib import colormaps
import matplotlib.patches as mpatches
from unidecode import unidecode


# Normalize name
def normalize_str(text: str) -> str:
    return unidecode(text).upper()


# Show the most prevalent issue category in each municipality
def plot_most_prevalent_issue_by_municipality(
        clean_df: pd.DataFrame,
        category_level: int = 2
):
    issue_by_municipality = {}
    for municipality in clean_df["Concelho"].unique():
        clean_df_by_municipality = clean_df[
            clean_df["Concelho"] == municipality
        ]
        most_prevalent_issue = clean_df_by_municipality.mode()[
            f"category{category_level}"
        ][0]
        issue_by_municipality[municipality] = most_prevalent_issue

    # Declare the color map that will be used
    color_map_names = ["viridis", "plasma", "cividis"]
    base_color_map = colormaps[color_map_names[category_level - 1]].resampled(
        len(set(issue_by_municipality.values()))
    )
    color_by_issue = {
        issue_name: base_color_map.colors[idx]
        for idx, issue_name in enumerate(set(issue_by_municipality.values()))
    }

    # Get shape of municipalities and parishes
    portugal_municipalities = gpd.read_file(
        "../georeferencing/data_concelhos/concelhos.shp"
    )
    # Remove islands
    portugal_municipalities = portugal_municipalities[
        portugal_municipalities.NAME_1 != "Azores"
    ]
    portugal_municipalities = portugal_municipalities[
        portugal_municipalities.NAME_1 != "Madeira"
    ]
    # Normalize municipality name
    portugal_municipalities["NAME_2"] = (
        portugal_municipalities["NAME_2"].astype(str).apply(normalize_str)
    )

    pt_map = portugal_municipalities.plot(color="gray")

    for municipality_name, issue_name in issue_by_municipality.items():
        matching_municipalities = portugal_municipalities.query(
            f"NAME_2=='{normalize_str(municipality_name)}'"
        )
        for _, row in matching_municipalities.iterrows():
            municipality_gdf = gpd.GeoDataFrame([row], crs=portugal_municipalities.crs)
            municipality_gdf.plot(ax=pt_map, color=color_by_issue[issue_name])

    pt_map.legend(
        handles=[
            # mpatches.Patch(color="gray", label="Sem dados"),  # We have all of it!
            *[
                mpatches.Patch(color=base_color_map.colors[idx], label=issue_name)
                for idx, issue_name in enumerate(set(issue_by_municipality.values()))
            ],
        ],
        loc="lower left",
        bbox_to_anchor=(0.1, 1),
    )
    plt.axis("off")
    plt.show()

Finally, we created a random forest classifier to evaluate the probability of each occurrence on a specific date in each municipality.

In [ ]:
dataset = pd.read_csv("Data/anpc-2016.csv")

In [ ]:
occurrences = parse_categories(clean_data(dataset))

In [ ]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder, OneHotEncoder
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score

# Get relevant and error-free data
occurrences = parse_categories(clean_data(occurrences))

occurrences["day_idx"] = occurrences["DataOcorrencia"] - min(
    occurrences["DataOcorrencia"]
)
occurrences["day_idx"] = occurrences["day_idx"].apply(
    lambda occ: occ.total_seconds() / 3600
)

# encode the categorical variables
le_state = LabelEncoder()
region = "Distrito"
occurrences[region] = le_state.fit_transform(occurrences[region])
enc = OneHotEncoder(sparse_output=False, handle_unknown="ignore")
encoded_data = enc.fit_transform(occurrences[[region]])

# combine the encoded variables with the numerical variables
X = pd.concat(
    [pd.DataFrame(encoded_data).reset_index(inplace=True), occurrences["day_idx"]],
    axis=1,
)
print(X)
y = occurrences["category2"]

# split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=12
)

# train a random forest classifier
clf = RandomForestClassifier(n_estimators=1000, random_state=12)
clf.fit(X_train, y_train)

# make predictions on the test set
y_pred = clf.predict(X_test)

# calculate the accuracy score
accuracy = accuracy_score(y_test, y_pred)
print(f"Accuracy score: {accuracy:.2f}")
